In [1]:
# Aperture photometry with photutils



In [2]:
# Packages

from astroquery.vizier import Vizier
import astropy.units as u
import astropy.coordinates as coord
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
from matplotlib import pyplot as plt
from numpy import sin, cos, arctan, degrees, radians, sqrt
from photutils.aperture import aperture_photometry
from photutils.aperture import CircularAperture

%matplotlib widget

In [3]:
def get_file(fname):
    # Достает из файла data и header
    hdu    = fits.open(fname)
    data   = hdu[0].data
    header = hdu[0].header
    return data, header
#-------------------------------------------------------------------
def angular_distance(ra, dec, RA, DEC):
    # находит угловое расстояние между двумя экв. координат в радианах!
    vec1 = [cos(ra)*cos(dec), sin(ra)*cos(dec), sin(dec)]
    vec2 = [cos(RA)*cos(DEC), sin(RA)*cos(DEC), sin(DEC)]
    return np.arccos(np.dot(vec1, vec2))
#-------------------------------------------------------------------
def get_image_center(filelist):
    max_radius = 0
    ras    = []
    decs   = []
    # Находит средний центр и радиус по всем кадрам в градусах
    for fname in filelist:
        #print(fname)
        imdata, header = get_file(fname)
        try:
            w = header['IMAGEW']
            h = header['IMAGEH']
        except:
            h, w = imdata.shape
        W = WCS(header)
        # экваториальные координаты центра кадра в градусах
        RAc, DECc = W.wcs_pix2world(np.array([[w/2, h / 2]], np.float_), 1)[0]
        RA1, DEC1 = W.wcs_pix2world(np.array([[0, 0]], np.float_), 1)[0]
        RA2, DEC2 = W.wcs_pix2world(np.array([[w, h]], np.float_), 1)[0]
        radius = degrees(angular_distance(radians(RA1), radians(DEC1), 
                                            radians(RA2), radians(DEC2)))/2
        if radius > max_radius:
            max_radius = radius
        decs.append(DECc)
        ras.append(RAc)

    #print('All Racs = ', ras)
    #print('All Decs = ', decs)
    RAc_mean  = np.mean(ras)
    DECc_mean = np.mean(decs)
    
    print('DECc = ', DECc_mean)
    print('RAc  = ', RAc_mean)
    print('radius = %s min' %(max_radius*60))
    return RAc_mean, DECc_mean, radius

#-----------------------------------------------------------------
def get_stars_coords(df, W, bdr, w, h):
    pxpos = W.wcs_world2pix(np.stack((df['ra'], df['dec']), axis=-1),1)
    # отбираем только звезды в пределах кадра с учетом поребрика
    p_indx = np.where((pxpos[:,0]>bdr) & (pxpos[:,1]>bdr) & (pxpos[:,0]<w-bdr) & (pxpos[:,1]<h-bdr))
    # перекидываем их в отдельный массив xy
    xy = pxpos[p_indx]
    mags   = np.asarray(df['phot_g_mean_mag'])[p_indx]
    # отбираем в массивы экваториальных координат только звезды в пределах кадра (с индексами p_indx)
    ra, dec = np.asarray(df.iloc[p_indx]['ra']),np.asarray(df.iloc[p_indx]['dec'])
    
    return xy, mags, ra, dec
#-------------------------------------------------------------------

def load_stars_from_Vizir(RAc, DECc, rc, low_mag, up_mag, 
                          out_file=None, catalog=['NOMAD'], filters={}):
    
    Vizier.ROW_LIMIT = -1
    RAc = RAc * u.degree
    DECc = DECc * u.degree
    radius = rc * u.degree
    result = Vizier.query_region(coord.SkyCoord(ra=RAc, dec=DECc, frame='icrs'),radius=radius,catalog=catalog,  column_filters={'Vmag': '%s : %s' %(low_mag, up_mag)} )
    df = result[0].to_pandas()
    if out_file is None:
        out_file = 'stadarts.csv'

    df.to_csv(out_file)
    return df
    
#--------------------------------------------------------------------
def get_stars_for_phot(df, W, bdr, w, h):
    pxpos = W.wcs_world2pix(np.stack((df['RAJ2000'], df['DEJ2000']), axis=-1),1)
    #pxpos = W.wcs_world2pix(np.stack((df['ra'], df['dec']), axis=-1),1)

    # отбираем только звезды в пределах кадра с учетом поребрика
    p_indx = np.where((pxpos[:,0]>bdr) & (pxpos[:,1]>bdr) & (pxpos[:,0]<w-bdr) & (pxpos[:,1]<h-bdr))
    # перекидываем их в отдельный массив xy
    xy = pxpos[p_indx]
    #Bmag   = np.asarray(df['phot_bp_mean_mag'])[p_indx]#np.asarray(df['Bmag'])[p_indx]
    #Vmag   = np.asarray(df['phot_g_mean_mag'])[p_indx]#np.asarray(df['Vmag'])[p_indx]
    #Rmag   = np.asarray(df['phot_rp_mean_mag'])[p_indx]#np.asarray(df['Rmag'])[p_indx]
    
    Bmag   = np.asarray(df['Bmag'])[p_indx]
    Vmag   = np.asarray(df['Vmag'])[p_indx]
    Rmag   = np.asarray(df['Rmag'])[p_indx]
    
    
    # Выкидывкем звезды, для которых нет данных каталога
    inds = np.where(~ (np.isnan(Bmag) + np.isnan(Vmag) + np.isnan(Rmag)))
    xy = xy[inds]
    Bmag   = Bmag[inds]
    Vmag   = Vmag[inds]
    Rmag   = Rmag[inds]
        
    return xy, Bmag, Vmag, Rmag

#------------------------------------------------------------------------------
def plot_sky(ima, cf1=0.1, cf2=1.6, xy=None, asteroid=None):
    cf1 = cf1# определяет окно значений отсчетов, которые будут отрисованы (смотри plt.imshow дальше в этом блоке) 
    cf2 = cf2# определяет окно значений отсчетов, которые будут отрисованы (смотри plt.imshow дальше в этом блоке) 

    fig, ax = plt.subplots(figsize=(3.0,3.0), dpi=300)
    ax.xaxis.set_tick_params(labelsize=5)
    ax.yaxis.set_tick_params(labelsize=5)
    plt.xlabel('X, pix', fontsize = 4)
    plt.ylabel('Y, pix', fontsize=4)
    #рисуем изображение в серой шкале. Можно поменять cmap='gray' на что-нибудь еще и можно получить всякие более модные отображения
    plt.imshow(ima, vmin=np.median(ima)- cf1*np.std(ima) ,
           vmax=np.median(ima)  + cf2*np.std(ima), origin='lower', cmap='gray')
    if xy is None:
        pass
    else:
        for p in xy:
            plt.scatter(p[0], p[1], s=7, facecolors='none', edgecolors='g', linewidth=0.4)
        #plt.scatter(y_a, x_a, s=7, facecolors='none', edgecolors='r', linewidth=0.4)    
    plt.tight_layout()
    plt.show()
    plt.clf()
    plt.close()

    
def del_nearest(xs, ys, ap):
    # Функция, убирающая из массива тесные звезды

    goods = []
    i = 0
    ind = []
    for x, y in zip(xs, ys):
        dist = np.sort(np.hypot(xs - x, ys - y))[1]
        if dist > ap:
            goods.append([x, y])
            ind.append(i)
        i += 1
    return goods, ind    

from astroquery.gaia import Gaia
import ssl
import pandas as pd
def get_stars_from_GAIA(RAc, DECc, limmag1, limmag2, radius, filename=None):
    # Фунция отбирает звезды на кадре. Возвращает их координаты и зв. величины
    #отбираем звезды из Gaia DR3 с блеском от limmag1 до limmag2 и сохраняем данные в csv-файл
    limmag1,limmag2 = limmag1, limmag2
    ssl._create_default_https_context = ssl._create_unverified_context
    fov = radius # это размер области в градусах
    if filename is None:
        filename = 'Gaia_cur.csv'
    # список звезд сохраняется в файл BP_PSC_Gaia.csv
    job = Gaia.launch_job_async("SELECT * FROM gaiadr3.gaia_source \
        WHERE CONTAINS(POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),CIRCLE('ICRS',%f,%f,%f))=1\
        AND  phot_g_mean_mag>%f AND  phot_g_mean_mag<%f;"\
                %(RAc,DECc,fov,limmag1,limmag2), dump_to_file=True, output_format='csv',
                            output_file=filename)
    df = pd.read_csv(filename)
    return df

In [4]:
from photutils.aperture import CircularAnnulus, CircularAperture
from photutils.aperture import ApertureStats, CircularAperture
from astropy.visualization import simple_norm
from photutils.utils import calc_total_error

def grow_curve(data, xs, ys, gal):
    plt.figure()
    plt.title(gal)
    for x, y  in zip(xs, ys):
        aps = np.arange(3, 20, 1)
        apertures = [CircularAperture((x,y), r=r) for r in aps ]
        phot_table = aperture_photometry(data, apertures)
        f = [phot_table['aperture_sum_%s' %i] for i in range(len(aps))]
        
        plt.plot(aps, f/np.max(f), '-o', label='(%5.2f, %5.2f)' %(x, y))
    plt.xlabel('Aper, pix')
    plt.ylabel('$F/F_{max}$')
    #plt.legend()
    #plt.clf()
    #plt.close()
    plt.show()


def aperture(data, xy, fwhm=9.0, plot=False):
    positions = xy
    aperture = CircularAperture(positions, r=2*fwhm)
    annulus_aperture = CircularAnnulus(positions, r_in=2.5*fwhm, r_out=3*fwhm)
    from astropy.stats import SigmaClip
    sigclip = SigmaClip(sigma=3.0, maxiters=10)
    effective_gain = 300
    error = calc_total_error(data, np.zeros(data.shape), effective_gain)
     
    aperstats_star = ApertureStats(data, aperture, sigma_clip=None, error=error)
    aperstats_bkg = ApertureStats(data, annulus_aperture, sigma_clip=sigclip)

    
    total_bkg = aperstats_bkg.median * aperstats_star.sum_aper_area.value
    flux = aperstats_star.sum - total_bkg

    #for a, b in zip(aperstats_star.sum, total_bkg ):
    #    print(a, b)
    mag =  -2.5*np.log10(flux) 
    #tab = aperstats_star.to_table()
    #tab.write('tm.csv', overwrite=True)
    #tab = aperstats_bkg.to_table()
    #tab.write('tmb.csv', overwrite=True)
    #print(aperstats_bkg.to_tabe())
    if plot:
        plt.figure(figsize=(10,10))
        norm = simple_norm(data, 'sqrt', percent=99)
        plt.imshow(data, norm=norm, interpolation='nearest')

        ap_patches = aperture.plot(color='white', lw=2,
                               label='Photometry aperture')
        ann_patches = annulus_aperture.plot(color='red', lw=2,
                                        label='Background annulus')
        handles = (ap_patches[0], ann_patches[0])
        plt.legend(loc=(0.17, 0.05), facecolor='#458989', labelcolor='white',handles=handles, prop={'weight': 'bold', 'size': 11})
        plt.show()
    return mag

In [5]:
def move_phot_sys(fnameB, fnameV, fnameR, pb0, pb1, pb2):
    hdu = fits.open(fnameB)
    dataB = -2.5*np.log10(hdu[0].data)
    headerB = hdu[0].header

     
    hdu = fits.open(fnameV)
    dataV = -2.5*np.log10(hdu[0].data)
    headerV = hdu[0].header
    

    hdu = fits.open(fnameR)
    dataR = -2.5*np.log10(hdu[0].data)
    headerR = hdu[0].header

    c0  = pb0[0]
    cv  = pb0[1]
    c1  = 1/pb1[0]
    cbv = -pb1[1]/c1
    c2  = 1/pb2[0]
    cvr = -pb2[1]/c2

    
    print('c0', c0)
    print('c1', c1)
    print('c2', c2)
    print('cbv', cbv)
    print('cvr', cvr)
    print('cv', cv)

    V = dataV + c0*(dataB-dataV) + cv
    BV = c1*(dataB-dataV) + cbv
    VR = c2*(dataV-dataR) + cvr

    B = BV + V
    R = V - VR

    Iv = 10**(-0.4*V)
    Iv[np.where(np.isnan(Iv))] = 0
    Ib = 10**(-0.4*B)
    Ib[np.where(np.isnan(Ib))] = 0
    Ir = 10**(-0.4*R)
    Ir[np.where(np.isnan(Ir))] = 0
    

    nameB = fnameB.split('/')[-1]
    fits.writeto(nameB, Ib, header=headerB, overwrite=True) 

    nameV = fnameV.split('/')[-1]
    fits.writeto(nameV, Iv, header=headerV, overwrite=True) 

    nameR = fnameR.split('/')[-1]
    fits.writeto(nameR, Ir, header=headerR, overwrite=True) 
    return

def linear(B, x):
    return B[0]*x + B[1]



def get_equals(cat_B, cat_V, cat_R, est_B, est_V, est_R, fnameB, fnameV, fnameR):

    gal = fnameB.split('/')[-1].split('-')[0]
    #plot_sky(ima, 0.1, 0.1, xy=list(zip(xB, yB)), xy1=list(zip(xV, yV)), xy2=list(zip(xR, yR)))
    #plt.savefig(gal+'.png')
    
    inds = np.where( ~(np.isnan(est_B)))
    est_B = est_B[inds]
    est_V = est_V[inds]
    est_R = est_R[inds]
    
    cat_B = cat_B[inds]
    cat_V = cat_V[inds]
    cat_R = cat_R[inds]
    
    
    def lin(x, *p):
        return p[0]*x + p[1]

    def lin0(x, *p):
        return x + p[0]
    
    def lin00(x, *p):
        return p[0]
    
    bv = est_B - est_V
    BV = cat_B - cat_V
    

    vr = est_V - est_R
    VR = cat_V - cat_R
    
    Vv = cat_V - est_V 
    
    inds = np.where( (-1 <VR) * (VR < 1)* (BV < 2))
    BV = BV[inds]
    bv = bv[inds]
    VR = VR[inds]
    vr = vr[inds]
    Vv = Vv[inds]
    
    from scipy.optimize import curve_fit
    pb10, covpb = curve_fit(lin0, xdata=BV, ydata=bv, p0=np.array([0])) #, bounds=np.array([[0.6, -np.inf], [1.5,np.inf]]))
    sy = abs(BV + pb10[0] - bv)
   
    for _ in range(1):
        pb1, covpb = curve_fit(lin, xdata=BV, ydata=bv, p0=np.array([1,0]), sigma=sy) #, bounds=np.array([[0.6, -np.inf], [1.5,np.inf]]))
        sy = abs(BV + pb1[1] - bv)
        
    s = pb1[0]
    i = pb1[1]
    s_err = np.sqrt(covpb[0,0])
    i_err = np.sqrt(covpb[0,0])
    a, b = -0.5, 2#inp.min(est_B - est_V), np.max(est_B-est_V)
    plt.figure(dpi=300)
    plt.title(gal)
    plt.plot(BV, bv, 'o')
    plt.plot([a,b], [a*s+i, b*s+i], '-r' , label=r'$c_1 = %5.3f \pm %5.3f$ $c_{bv} = %5.3f \pm %5.3f$' %(s, s_err, i, i_err))
    plt.xlabel('B-V')
    plt.ylabel('b-v')
    plt.legend()
    plt.xlim(0, 2)
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.savefig(gal+'BV.png')
    plt.clf()
    plt.close()
    #plt.show()
    
    pb20, covpb = curve_fit(lin0, xdata=VR, ydata=vr, p0=np.array([0])) #, bounds=np.array([[0.6, -np.inf], [1.5,np.inf]]))
    sy = abs(VR + pb20[0] - vr)    
    for _ in range(1):
        pb2, covpb = curve_fit(lin, xdata=VR, ydata=vr, p0=np.array([1,0]), sigma=sy)#, bounds=np.array([[0.6, -np.inf], [1.5,np.inf]]))
        sy = abs(VR*pb2[0] + pb2[1] - vr)
    s = pb2[0]
    i = pb2[1]
    s_err = np.sqrt(covpb[0,0])
    i_err = np.sqrt(covpb[1,1])
    a, b = np.min(cat_V - cat_R), np.max(cat_V-cat_R)
    plt.figure(dpi=300)
    plt.title(gal)
    plt.plot(VR, vr, 'o')
    plt.plot([a,b], [a*s+i, b*s+i], '-r', label=r'$c_2 = %5.3f \pm %5.3f$ $c_{vr} = %5.3f \pm %5.3f$' %(s, s_err, i, i_err))
    plt.xlabel('V-R')
    plt.ylabel('v-r')
    plt.legend()
    plt.xlim(-2, 2)
    #plt.ylim(-2, 2)
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.savefig(gal+'VR.png')
    plt.clf()
    plt.close()
    #plt.show()

    pb00, covpb = curve_fit(lin00, xdata=bv, ydata=Vv, p0=np.array([0])) #, bounds=np.array([[0.6, -np.inf], [1.5,np.inf]]))
    sy = abs(pb00[0] - Vv) 
    
    for _ in range(1):
        pb0, covpb = curve_fit(lin, xdata=bv, ydata=Vv, p0=np.array([0,0]), sigma=sy)
        sy = abs(pb0[0]*bv*0 + pb0[1] - Vv) 

    s = pb0[0]
    i = pb0[1]
    s_err = np.sqrt(covpb[0,0])
    i_err = np.sqrt(covpb[1,1])
    ia = np.argmin(bv)
    ib = np.argmax(bv)
    
    BV = est_B - est_V
    _BV = np.array([])
    _V = np.array([]) 
    for b, a in sorted(zip(est_V, BV)):
        _BV = np.append(_BV, a)
        _V = np.append(_V, b)

    
    plt.figure(dpi=300)
    plt.title(gal)
    plt.plot(bv, Vv,'o')
    plt.plot(_BV, _BV*s + i, '-r', label=r'$c_0 = %5.3f \pm %5.3f$ $c_v = %5.3f \pm %5.3f$' %(s, s_err, i, i_err))
    plt.xlabel('b - v')
    plt.ylabel('V - v')
    plt.legend()
    
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.savefig(gal+'V.png')
    plt.clf()
    plt.close()
    #plt.show()

    #plt.figure()
    #plt.plot(cat_B - cat_V, cat_V - cat_R, 'o')
    #plt.show()
    
    move_phot_sys(fnameB, fnameV, fnameR, pb0, pb1, pb2)

In [6]:
def main(fnames, bdr=200):
    RAc, DECc, rc = get_image_center(fnames)
    #df = get_stars_from_GAIA(RAc, DECc, limmag1=16, limmag2=18, radius=rc, filename=None)
    df = load_stars_from_Vizir(RAc, DECc, rc, low_mag=14, up_mag=17.5, 
                          out_file=None, catalog=['NOMAD'], filters={})
    
    mags = []
    for fname in fnames:
        data, header = get_file(fname)
        gal = fname.split('-')[0].split('/')[-1]
        if gal == 'UGC9560':
            gal += '-62'
        elif gal == 'NGC5430':
            gal += '_new'
        mask, _ = get_file('WMO/%s/mask.fits' %gal)

        mask = mask == 1
        
        data[mask] = float('nan')
        
        
        h, w = data.shape
        W = WCS(header)
        xy, Bmag, Vmag, Rmag = get_stars_for_phot(df, W, bdr, w, h)

        xs = [x[0] for x in xy]
        ys = [x[1] for x in xy]
        #grow_curve(data, xs, ys, gal)
        #plot_sky(data, 0.01, 0.01, xy)
        #grow_curve(data, xs, xs)
        mag = aperture(data, xy, fwhm=9)
        #print(len(mag))
        mags.append(mag)
        
    Best, Vest, Rest = mags
    
    #Best += 2.5*np.log10(300)
    #Vest += 2.5*np.log10(300)
    #Rest += 2.5*np.log10(300)
    print(gal)
    get_equals(cat_B=Bmag, cat_V=Vmag, cat_R=Rmag, est_B=Best, est_V=Vest, est_R=Rest, fnameB=fnames[0], fnameV=fnames[1], fnameR=fnames[2])
    
    '''
    ind = np.where((Bmag - Vmag) > 1.5)
    xy = np.array(xy)
    plot_sky(data, 0.1, 0.1, xy=xy[ind])
    plt.figure()
    plt.plot(Best - Vest, Bmag -  Vmag, 'o')
    
    plt.show()

    plt.figure()
    plt.plot(Vest - Rest, Vmag -  Rmag, 'o')
    #plt.xlim(-2,2)
    plt.show()
    
    plt.figure()
    plt.plot(Best - Vest, Vmag -  Vest, 'o')
    plt.show()
    '''
    

In [7]:
gals = ['UGC9560','NGC7743','NGC6181','NGC5660','NGC5430','NGC5371','NGC3583','NGC895' ,'M100']
for gal in gals:
    main(['WMO/bkg_est_result/%s-B.fits' %gal, 'WMO/bkg_est_result/%s-V.fits' %gal,'WMO/bkg_est_result/%s-R.fits' %gal])

DECc =  35.56041332144716
RAc  =  222.78472458439327
radius = 14.37448335407398 min


/tmp/ipykernel_102027/4163833808.py:42: RuntimeWarning: invalid value encountered in log10
  mag =  -2.5*np.log10(flux)
/tmp/ipykernel_102027/4163833808.py:42: RuntimeWarning: invalid value encountered in log10
  mag =  -2.5*np.log10(flux)


UGC9560-62


/tmp/ipykernel_102027/3652196351.py:3: RuntimeWarning: divide by zero encountered in log10
  dataB = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:3: RuntimeWarning: invalid value encountered in log10
  dataB = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:8: RuntimeWarning: divide by zero encountered in log10
  dataV = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:8: RuntimeWarning: invalid value encountered in log10
  dataV = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:13: RuntimeWarning: divide by zero encountered in log10
  dataR = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:13: RuntimeWarning: invalid value encountered in log10
  dataR = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:31: RuntimeWarning: invalid value encountered in subtract
  V = dataV + c0*(dataB-dataV) + cv
/tmp/ipykernel_102027/3652196351.py:32: RuntimeWarning: invalid value encountered in subtract
  BV = c1

c0 -0.017628718215599658
c1 1.1340279093481669
c2 1.0950380998542666
cbv -0.23137138446136357
cvr -0.4640176512644714
cv 28.766637867594632


...

c0 0.0025052335446984486
c1 1.0138728931465852
c2 0.9894684504289181
cbv -0.020012715646207206
cvr -0.14283612761499312
cv 28.76233469415594
DECc =  -5.541092291611402
RAc  =  35.40515365047671
radius = 14.38606882462154 min
NGC895


/tmp/ipykernel_102027/3652196351.py:3: RuntimeWarning: divide by zero encountered in log10
  dataB = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:3: RuntimeWarning: invalid value encountered in log10
  dataB = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:8: RuntimeWarning: invalid value encountered in log10
  dataV = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:13: RuntimeWarning: divide by zero encountered in log10
  dataR = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:13: RuntimeWarning: invalid value encountered in log10
  dataR = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:35: RuntimeWarning: invalid value encountered in add
  B = BV + V
/tmp/ipykernel_102027/3652196351.py:36: RuntimeWarning: invalid value encountered in subtract
  R = V - VR


c0 -0.0036598618560548246
c1 0.9964832476546319
c2 1.1866814455789871
cbv -0.28473146476658506
cvr -0.5978259958456109
cv 28.292616381673074
DECc =  15.815282753379185
RAc  =  185.72181542493846
radius = 14.376550524507817 min
M100


/tmp/ipykernel_102027/3652196351.py:3: RuntimeWarning: divide by zero encountered in log10
  dataB = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:3: RuntimeWarning: invalid value encountered in log10
  dataB = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:8: RuntimeWarning: divide by zero encountered in log10
  dataV = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:8: RuntimeWarning: invalid value encountered in log10
  dataV = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:13: RuntimeWarning: divide by zero encountered in log10
  dataR = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:13: RuntimeWarning: invalid value encountered in log10
  dataR = -2.5*np.log10(hdu[0].data)
/tmp/ipykernel_102027/3652196351.py:33: RuntimeWarning: invalid value encountered in subtract
  VR = c2*(dataV-dataR) + cvr
/tmp/ipykernel_102027/3652196351.py:35: RuntimeWarning: invalid value encountered in add
  B = BV + V


c0 -0.02613719735660247
c1 0.9900577069548347
c2 0.9817588687688238
cbv -0.008477383078793456
cvr -0.45848112319799944
cv 28.648658987848677
